In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 38.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 47.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 4.5 MB/s eta 0:00:00


In [2]:
from transformers import pipeline

문자 토큰화

In [7]:
text = "Tokenizing text is a core task of NLP."
tokenized_text = list(text)
print(tokenized_text)

['T', 'o', 'k', 'e', 'n', 'i', 'z', 'i', 'n', 'g', ' ', 't', 'e', 'x', 't', ' ', 'i', 's', ' ', 'a', ' ', 'c', 'o', 'r', 'e', ' ', 't', 'a', 's', 'k', ' ', 'o', 'f', ' ', 'N', 'L', 'P', '.']


In [8]:
token2idx = {ch: idx for idx, ch in enumerate(sorted(set(tokenized_text)))}
print(token2idx)

{' ': 0, '.': 1, 'L': 2, 'N': 3, 'P': 4, 'T': 5, 'a': 6, 'c': 7, 'e': 8, 'f': 9, 'g': 10, 'i': 11, 'k': 12, 'n': 13, 'o': 14, 'r': 15, 's': 16, 't': 17, 'x': 18, 'z': 19}


In [9]:
input_ids = [token2idx[token] for token in tokenized_text] 
print(input_ids)

[5, 14, 12, 8, 13, 11, 19, 11, 13, 10, 0, 17, 8, 18, 17, 0, 11, 16, 0, 6, 0, 7, 14, 15, 8, 0, 17, 6, 16, 12, 0, 14, 9, 0, 3, 2, 4, 1]


원핫벡터 인코딩(one-hot-vector):문자 토큰화 예

In [10]:
import torch
import torch.nn.functional as F

input_ids = torch.tensor(input_ids)
one_hot_encodings = F.one_hot(input_ids, num_classes=len(token2idx))
#one hot 함수가 있다
one_hot_encodings.shape

torch.Size([38, 20])

In [11]:
print(f"토큰:{tokenized_text[11]}")
print(f"텐서 인덱스: {input_ids[11]}")
print(f"원-핫 인코딩: {one_hot_encodings[11]}")

토큰:t
텐서 인덱스: 17
원-핫 인코딩: tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])


단어 토큰화

공백 기준 토큰화(전통적인 NLP 방법)
* 구두점, 어간, 어미 정규화 등 확장 가능

장점
* 학습의 복잡도 감소 : 모델이 문자에서 단어를 학습

단점
* 토큰~id 사전(어휘 사전)의 크기 문제
* 1백만 유니크 단어라면?


> 1 백만 유니크 단어를 1000차원의 벡터로 인코딩하는 행렬
> 1000M 의 가중치 필요

* 어휘사전 구축 시 발견되지 않았던 단어 처리 문제





부분 토큰화(subword tokenization)

WordPiece 토크나이저

* BPE 를 기반으로 하면서 문자열을 결합할 때 확률적인 방식을 사용한다. 즉, 가장 빈번하게 등장하는 문자열만 결합하는 것이 아니라, 조금 더 드물게 등장하는 문자열도 결합할 수 있도록 확률 분포를 이용한다. 
* [CLS] [SEP] 시퀀스의 시작과 끝을 알림, 소문자로 변환됨, ##izing 앞의 문자열이 공백이 아님을 의미(앞의 토큰과 합쳐져야 함)

In [12]:
from transformers import AutoTokenizer
#AutoTokenizer: WordPiece, BPE 등 모델마다 다양한 토그나이저를 일관되게 다룰 수 있는 자동 클래스
#로딩된 내용은 로컬 캐시에 저장됨

In [13]:
model_ckpt = "distilbert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [14]:
#토크나이징
#input_ids: 토큰의 고유 정수값들
encoded_text = tokenizer(text)
print(encoded_text)

{'input_ids': [101, 19204, 6026, 3793, 2003, 1037, 4563, 4708, 1997, 17953, 2361, 1012, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [16]:
#id -> 토큰
tokens = tokenizer.convert_ids_to_tokens(encoded_text.input_ids)
print(tokens)

['[CLS]', 'token', '##izing', 'text', 'is', 'a', 'core', 'task', 'of', 'nl', '##p', '.', '[SEP]']


In [17]:
#토큰->문자열
print(tokenizer.convert_tokens_to_string(tokens))

[CLS] tokenizing text is a core task of nlp. [SEP]


In [18]:
#어휘 사전 크기
tokenizer.vocab_size

30522

최대 컨텍스트

자연어 처리에서 하나의 단어나 문장을 처리하기 위해 고려 할 수 있는 이전과 이후의 단어나 문장의 개수를 의미한다

최대 컨덱스트의 크기는 매우 중요한 역할을 한다.

예를 들어 자연어 생성 모델에서는 이전의 문장을 고려하여 현재 문장을 생성하는 것이 중요하며, 자연어 이해 모델에서는 주어진 문장 내의 단어들 사이의 관계를 파악하기 위해 이전과 이후의 단어를 함께 고려해야 한다.

하지만 그렇다고 너무 크게 지정하면 모델의 계산량이 증가하고 학습 시간이 늘어나는 문제가 발생할 수 있다. 따라서, 최대 컨텍스트 크기는 모델의 성능과 계산 효율성 간의 균형을 고려하여 적잘한 크기로 설정하는 것이 좋다

In [19]:
#최대 컨텍스트 크기
tokenizer.model_max_length

512

In [20]:
#모델이 기대하는 입력 필드 이름
tokenizer.model_input_names

['input_ids', 'attention_mask']

In [24]:
#스페셜 토큰 정보
import pandas as pd
tokens2ids = list(zip(tokenizer.all_special_tokens, tokenizer.all_special_ids))
data = sorted(tokens2ids, key=lambda x:x[-1])
df = pd.DataFrame(data, columns=["Special Token","Special Token ID"])
df.T

,0,1,2,3,4
Special Token,[PAD],[UNK],[CLS],[SEP],[MASK]
Special Token ID,0,100,101,102,103


[분류 모델: 트랜스포머를 특성추출기로 활용]

1. 사전훈련된 모델 로딩
2. 마지막 은닉 상태 추출하기
3 사이킷런 스타일로 훈련
4. 간단한 분류 모델 훈련하기: logistic regression

In [26]:
from transformers import AutoModel

model_ckpt = "distilbert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#GPU/CPU 선택 가능
model = AutoModel.from_pretrained(model_ckpt).to(device)
#주요 기능: 토큰 인코딩 -> 은닉 상태를 반환

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [28]:
text = "this is a test"
inputs = tokenizer(text, return_tensors = "pt")
print(f"입력 텐서 크기: {inputs['input_ids'].size()}")

입력 텐서 크기: torch.Size([1, 6])


In [29]:
inputs = {k:v.to(device) for k,v in inputs.items()}
with torch.no_grad():
  outputs = model(**inputs)
  
print(outputs)

BaseModelOutput(last_hidden_state=tensor([[[-0.1565, -0.1862,  0.0528,  ..., -0.1188,  0.0662,  0.5470],
         [-0.3575, -0.6484, -0.0618,  ..., -0.3040,  0.3508,  0.5221],
         [-0.2772, -0.4459,  0.1818,  ..., -0.0948, -0.0076,  0.9958],
         [-0.2841, -0.3917,  0.3753,  ..., -0.2151, -0.1173,  1.0526],
         [ 0.2661, -0.5094, -0.3180,  ..., -0.4203,  0.0144, -0.2149],
         [ 0.9441,  0.0112, -0.4714,  ...,  0.1439, -0.7288, -0.1619]]]), hidden_states=None, attentions=None)


In [30]:
outputs.last_hidden_state.size()
#batchsize, token 개수, 토큰의 벡터 길이

torch.Size([1, 6, 768])

In [31]:
outputs.last_hidden_state[:,0].size()

torch.Size([1, 768])

In [37]:
def extract_hidden_states(batch):
  #모델 입력을 GPU로 옮깁니다
  inputs = {k:v.to(device) for k,v in batch.items() if k in tokenizer.model_input_names}
  #마지막 은닉 상태를 추출합니다
  with torch.no_grad():
    last_hidden_state = model(**inputs).last_hidden_state
    #[CLS] 토큰에 대한 벡터를 반환한다
  return {"hidden_state": last_hidden_state[:0].cpu().numpy()}

In [38]:
  emotions_encoded.set_format("torch",columns=["input_ids","attention_mask","label"])

NameError: ignored

In [34]:
emotions_hidden = emotions_encoded.map(extract_hidden_states,batched=True)

NameError: ignored

In [ ]:
 emotions_hidden["train"].column_names